In [ ]:
from binance.spot import Spot as Client
from binance.websocket.spot.websocket_stream import SpotWebsocketStreamClient
import time
import base64
import hashlib
import hmac
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
proxies = { 'https': 'http://127.0.0.1:1080', 'http': 'http://127.0.0.1:1080' }
client= Client(proxies=proxies, show_limit_usage=True)

In [ ]:
res = client.klines('BTCUSDT', interval='1m', limit=720)

In [ ]:
ba_df = pd.DataFrame(res['data'], columns=['date_open', 'open', 'high', 'low', 'close', 'vol', 'date_close', 'asset_vol', 'count', 'base_vol', 'quote_vol', 'ignore'])

ba_df['date_open'] = pd.to_datetime(ba_df['date_open'], unit='ms')
ba_df['date_close'] = pd.to_datetime(ba_df['date_close'], unit='ms')

In [ ]:
def message_handler(_, message):
    print(message)

my_client = SpotWebsocketStreamClient(on_message=message_handler, proxies=proxies, timeout=10)

# Subscribe to a single symbol stream
# my_client.agg_trade(symbol="bnbusdt")
my_client.kline(symbol='btcusdt', interval='1m')
time.sleep(5 * 60)
logging.info("closing ws connection")
my_client.stop()

In [ ]:
import requests
import json

url = "https://api.kraken.com/0/private/GetWebSocketsToken"


api_key = '3kZXu4AX+c+J2BALKrEo0RvWugnBu7xgSN8/Jz6ezP3RMPx0ikIE4pnd'
api_secret = '+jHEC+BvgLCQJzJ9yIFHW8eWdAPrcDIBc1s/sBG5hVWBRakEnBGnW4/tTC4xMfb0biIlqLB9nTNwRMwgpbPbGA=='

api_path = '/0/private/GetWebSocketsToken'
api_nonce = str(int(time.time()*1000))
api_post = 'nonce=' + api_nonce

# Cryptographic hash algorithms
api_sha256 = hashlib.sha256(api_nonce.encode('utf-8') + api_post.encode('utf-8'))
api_hmac = hmac.new(base64.b64decode(api_secret), api_path.encode('utf-8') + api_sha256.digest(), hashlib.sha512)

# Encode signature into base64 format used in API-Sign value
api_signature = base64.b64encode(api_hmac.digest())

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'API-Key': api_key,
  'API-Sign': api_signature
}

payload = json.dumps({
  "nonce": api_nonce
})

response = requests.request("POST", url, headers=headers, data=payload, proxies=proxies)

print(response.text)

In [ ]:
import requests

url = "https://api.kraken.com/0/public/OHLC?pair=btcusd&interval=1"

payload = {}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload, proxies=proxies)

In [ ]:
kk_df = pd.DataFrame(response.json()['result']['XXBTZUSD'], columns=['date', 'open', 'high', 'low', 'close', 'vwap', 'vol', 'count'])
kk_df['date'] = pd.to_datetime(kk_df['date'], unit='s')

In [ ]:
for col in ['open', 'high', 'low', 'close']:
    ba_df[col] = ba_df[col].astype(float)
    kk_df[col] = kk_df[col].astype(float)

df = ba_df.merge(kk_df, left_on='date_open', right_on='date', how='left', suffixes=('.ba', '.kk'))
df['diff'] = df['close.ba'] - df['close.kk']

diff_df = df[['date', 'diff']]

df[['date_open', 'close.ba', 'date', 'close.kk', 'diff']].tail(10)

In [ ]:
diff_df['diff'].abs().describe()
abnormal = diff_df['diff'].abs().quantile(0.99)

diff_df[diff_df['diff'].abs() > abnormal]

In [ ]:
plt.rcParams['figure.dpi'] = 300
plt.figure(figsize=(10, 6))
plt.plot(diff_df['date'], diff_df['diff'], label='diff')


plt.xticks(rotation=45)

plt.legend()
plt.grid(True)
plt.show()